In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import urllib.parse
from urllib.parse import urljoin
import csv

In [6]:
df["title"] = df["title"].str.replace(r"[^a-zA-Z0-9\s]", "", regex=True).str.lower()
df["author"] = df["author"].str.replace(r"[^a-zA-Z0-9\s]", "", regex=True).str.lower()
df

,title,author,url,description,subjects
0,frankenstein or the modern prometheus,mary wollstonecraft shelley,https://www.gutenberg.org/ebooks/84,Free kindle book and epub digitized and proofr...,[]
1,moby dick or the whale,herman melville,https://www.gutenberg.org/ebooks/2701,Free kindle book and epub digitized and proofr...,[]
2,romeo and juliet,william shakespeare,https://www.gutenberg.org/ebooks/1513,Free kindle book and epub digitized and proofr...,[]
3,pride and prejudice,jane austen,https://www.gutenberg.org/ebooks/1342,Free kindle book and epub digitized and proofr...,[]
4,alices adventures in wonderland,lewis carroll,https://www.gutenberg.org/ebooks/11,Free kindle book and epub digitized and proofr...,[]
...,...,...,...,...,...
2389,a woman of no importance,oscar wilde,https://www.gutenberg.org/ebooks/854,Free kindle book and epub digitized and proofr...,[]
2390,chinese,active 17th century erzeng yang,https://www.gutenberg.org/ebooks/24231,Free kindle book and epub digitized and proofr...,[]
2391,discovery of witches,active 16121618 thomas potts,https://www.gutenberg.org/ebooks/18253,Free kindle book and epub digitized and proofr...,[]
2392,history of the wars books v and vi the gothic war,procopius,https://www.gutenberg.org/ebooks/20298,Free kindle book and epub digitized and proofr...,[]


In [8]:
df.shape

(2394, 5)

In [9]:
API_KEY = "AIzaSyAVWQOFupr20cHiAeA0cfcjEFTi-VcFiH4AIzaSyAVWQOFupr20cHiAeA0cfcjEFTi-VcFiH4"

In [10]:
df["categories"] = None
df["avg_rating"] = None
df["ratings_count"] = None
df["publisher"] = None
df["published_date"] = None
df["page_count"] = None
df["isbn10"] = None
df["isbn13"] = None
df["language"] = None
df["description_long"] = None

#def search_google_books(title, author=None):

    query = title
    if author:
        query += f" {author}"

    #url = f"https://www.googleapis.com/books/v1/volumes?q={query}&maxResults=1&key={API_KEY}"

    r = requests.get(url).json()

    if "items" not in r:
        return None

    return r["items"][0]["volumeInfo"]

for i, row in df.iterrows():

    info = search_google_books(row["title"], row["author"])

    if info:
        df.at[i, "categories"] = info.get("categories")
        df.at[i, "avg_rating"] = info.get("averageRating")
        df.at[i, "ratings_count"] = info.get("ratingsCount")
        df.at[i, "publisher"] = info.get("publisher")
        df.at[i, "published_date"] = info.get("publishedDate")
        df.at[i, "page_count"] = info.get("pageCount")
        df.at[i, "language"] = info.get("language")
        df.at[i, "description_long"] = info.get("description")

        ids = info.get("industryIdentifiers")
        if ids:
            for id in ids:
                if id["type"] == "ISBN_10":
                    df.at[i, "isbn10"] = id["identifier"]
                if id["type"] == "ISBN_13":
                    df.at[i, "isbn13"] = id["identifier"]

    # Incremental saving
    if i % 100 == 0:
        print(f"Saving {i} records...")
        df.to_csv("gutenberg_enriched.csv", index=False)

    time.sleep(0.2)

Saving 0 records...
Saving 100 records...
Saving 200 records...
Saving 300 records...
Saving 400 records...
Saving 500 records...
Saving 600 records...
Saving 700 records...
Saving 800 records...
Saving 900 records...
Saving 1000 records...
Saving 1100 records...
Saving 1200 records...
Saving 1300 records...
Saving 1400 records...
Saving 1500 records...
Saving 1600 records...
Saving 1700 records...
Saving 1800 records...
Saving 1900 records...
Saving 2000 records...
Saving 2100 records...
Saving 2200 records...
Saving 2300 records...


In [27]:
for i, row in df.iterrows():
    if pd.notna(row['avg_rating']):
        continue  # já enriquecido pela API

    query = f"{row['title']} {row['author']}"
    url = f"https://www.googleapis.com/books/v1/volumes?q={query}&maxResults=1&key={API_KEY}"
    r = requests.get(url).json()

    if "items" in r:
        info = r["items"][0]["volumeInfo"]

        df.at[i, 'categories'] = ', '.join(info.get('categories', []))
        df.at[i, 'avg_rating'] = info.get('averageRating')
        df.at[i, 'ratings_count'] = info.get('ratingsCount')
        df.at[i, 'publisher'] = info.get('publisher')
        df.at[i, 'published_date'] = info.get('publishedDate')
        df.at[i, 'page_count'] = info.get('pageCount')

        ids = info.get('industryIdentifiers', [])
        for idd in ids:
            if idd['type'] == "ISBN_10":
                df.at[i, 'isbn10'] = idd['identifier']
            if idd['type'] == "ISBN_13":
                df.at[i, 'isbn13'] = idd['identifier']

    time.sleep(0.2)


In [113]:
df.sample(5)

,title,author,url,description,subjects,categories,avg_rating,ratings_count,publisher,published_date,page_count,isbn10,isbn13,language,description_long
887,financial crime and corruption,samuel vaknin,https://www.gutenberg.org/ebooks/22747,Free kindle book and epub digitized and proofr...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2170,the second jungle book,rudyard kipling,https://www.gutenberg.org/ebooks/1937,Free kindle book and epub digitized and proofr...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2205,changeling and other stories,donn byrne,https://www.gutenberg.org/ebooks/59892,Free kindle book and epub digitized and proofr...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
723,new atlantis,francis bacon,https://www.gutenberg.org/ebooks/2434,Free kindle book and epub digitized and proofr...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
266,on the duty of civil disobedience,henry david thoreau,https://www.gutenberg.org/ebooks/71,Free kindle book and epub digitized and proofr...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
subjects = [
    "fiction", "history", "biography", "fantasy", "romance", "mystery",
    "science fiction", "young adult", "business", "computers",
    "philosophy", "psychology", "technology", "education", "art",
    "health", "travel", "religion", "sports", "cookbook",
    "poetry", "drama", "horror", "self help", "thriller",
    "music", "crime", "politics", "war", "mythology",
    "economics", "kids", "parenting", "science", "mathematics",
    "astronomy", "fantasy epic", "detective", "ai", "programming"
]

books_api = []

for s in subjects:
    for start in range(0,2000,40):
        url = (
            f'https://www.googleapis.com/books/v1/volumes?q=subject:"{s}"'
            f'&startIndex={start}&maxResults=40&orderBy=newest&printType=books&key={API_KEY}'
        )
        r = requests.get(url).json()

        if "items" not in r:
            continue

        for item in r["items"]:
            books_api.append(item)

        time.sleep(0.15)

len(books_api)



1010

In [31]:
df_api = pd.json_normalize(books_api)

df_api = df_api.drop_duplicates(subset="volumeInfo.industryIdentifiers")
df_api.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 1009
Data columns (total 57 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   kind                                                 1000 non-null   object 
 1   id                                                   1000 non-null   object 
 2   etag                                                 1000 non-null   object 
 3   selfLink                                             1000 non-null   object 
 4   volumeInfo.title                                     1000 non-null   object 
 5   volumeInfo.authors                                   981 non-null    object 
 6   volumeInfo.publisher                                 726 non-null    object 
 7   volumeInfo.publishedDate                             997 non-null    object 
 8   volumeInfo.description                               862 non-null    obje

In [32]:
df_api = df_api.rename(columns={
    "volumeInfo.title": "title",
    "volumeInfo.authors": "author",
    "volumeInfo.categories": "categories",
    "volumeInfo.description": "description",
    "volumeInfo.publisher": "publisher",
    "volumeInfo.publishedDate": "published_date",
    "volumeInfo.pageCount": "page_count",
    "volumeInfo.averageRating": "avg_rating",
    "volumeInfo.ratingsCount": "ratings_count",
})

In [33]:
from ast import literal_eval

def extract_isbn(d):
    if isinstance(d, list):
        isbn10, isbn13 = None, None
        for x in d:
            if x["type"] == "ISBN_10": isbn10 = x["identifier"]
            if x["type"] == "ISBN_13": isbn13 = x["identifier"]
        return pd.Series([isbn10, isbn13])
    return pd.Series([None, None])

df_api[["isbn10", "isbn13"]] = df_api["volumeInfo.industryIdentifiers"].apply(extract_isbn)


In [34]:
df_api["author"] = df_api["author"].astype(str).str.lower().str.strip()


In [36]:
df_api = df_api[[
    "title","author","description","categories",
    "avg_rating","ratings_count","publisher",
    "published_date","page_count","isbn10","isbn13"
]]
df_api.head()

,title,author,description,categories,avg_rating,ratings_count,publisher,published_date,page_count,isbn10,isbn13
0,Taras Bulba,['nikolái v. gogol'],"Feroces, crueles, valientes y apasionados, los...",[Literary Criticism],NaN,NaN,Ediciones AKAL,2006-12-19,154.0,8446023709,9788446023708
1,Frankenstein,['mary shelley'],"Frankenstein; or, The Modern Prometheus is an ...",[Fiction],NaN,NaN,Seven Books,2025-10-10,214.0,940383742X,9789403837420
2,Arabian Nights,nan,NaN,[Fiction],NaN,NaN,Library of Alexandria,2001-01-01,744.0,1465529004,9781465529008
3,The History of the Peloponnesian War,['thucydides'],The History of the Peloponnesian War (Greek: Ἱ...,[Fiction],NaN,NaN,Otbebookpublishing,2021-01-13,738.0,3968656768,9783968656762
4,The Plague,['albert camus'],“Its relevance lashes you across the face.” —S...,[Fiction],4.5,14.0,Vintage,1991-05-07,322.0,0679720219,9780679720218


In [39]:
df_api.info()

,title,author,description,categories,avg_rating,ratings_count,publisher,published_date,page_count,isbn10,isbn13
0,Taras Bulba,['nikolái v. gogol'],"Feroces, crueles, valientes y apasionados, los...",[Literary Criticism],NaN,NaN,Ediciones AKAL,2006-12-19,154.0,8446023709,9788446023708
1,Frankenstein,['mary shelley'],"Frankenstein; or, The Modern Prometheus is an ...",[Fiction],NaN,NaN,Seven Books,2025-10-10,214.0,940383742X,9789403837420
2,Arabian Nights,nan,NaN,[Fiction],NaN,NaN,Library of Alexandria,2001-01-01,744.0,1465529004,9781465529008
3,The History of the Peloponnesian War,['thucydides'],The History of the Peloponnesian War (Greek: Ἱ...,[Fiction],NaN,NaN,Otbebookpublishing,2021-01-13,738.0,3968656768,9783968656762
4,The Plague,['albert camus'],“Its relevance lashes you across the face.” —S...,[Fiction],4.5,14.0,Vintage,1991-05-07,322.0,0679720219,9780679720218


In [38]:
df_api.isna().sum()

title               0
author              0
description       138
categories        101
avg_rating        859
ratings_count     859
publisher         274
published_date      3
page_count         24
isbn10            193
isbn13            169
dtype: int64

In [42]:
df_api["author"] = df_api["author"].apply(lambda x: x[0] if isinstance(x, list) else x)
df_api["author"] = df_api["author"].str.lower().str.strip()



In [43]:
df_api["categories"] = df_api["categories"].apply(
    lambda x: ", ".join(x) if isinstance(x, list) else x
)


In [45]:
df_api["description"] = df_api["description"].fillna("")


In [49]:
df_api["author"] = df_api["author"].apply(
    lambda x: ", ".join(x) if isinstance(x, list) else str(x)
)

df_api["author"] = (
    df_api["author"]
    .str.replace(r"[\[\]']", "", regex=True)
    .str.lower()
    .str.strip()
)


In [55]:
df_api["author"] = df_api["author"].replace("nan", "unknown")


In [58]:
df_api["author"].apply(type).value_counts()
df_api.head()


,title,author,description,categories,avg_rating,ratings_count,publisher,published_date,page_count,isbn10,isbn13
0,Taras Bulba,nikolái v. gogol,"Feroces, crueles, valientes y apasionados, los...",Literary Criticism,NaN,NaN,Ediciones AKAL,2006-12-19,154.0,8446023709,9788446023708
1,Frankenstein,mary shelley,"Frankenstein; or, The Modern Prometheus is an ...",Fiction,NaN,NaN,Seven Books,2025-10-10,214.0,940383742X,9789403837420
2,Arabian Nights,unknown,,Fiction,NaN,NaN,Library of Alexandria,2001-01-01,744.0,1465529004,9781465529008
3,The History of the Peloponnesian War,thucydides,The History of the Peloponnesian War (Greek: Ἱ...,Fiction,NaN,NaN,Otbebookpublishing,2021-01-13,738.0,3968656768,9783968656762
4,The Plague,albert camus,“Its relevance lashes you across the face.” —S...,Fiction,4.5,14.0,Vintage,1991-05-07,322.0,0679720219,9780679720218


In [59]:
df.head()

,title,author,url,description,subjects,categories,avg_rating,ratings_count,publisher,published_date,page_count,isbn10,isbn13,language,description_long
0,frankenstein or the modern prometheus,mary wollstonecraft shelley,https://www.gutenberg.org/ebooks/84,Free kindle book and epub digitized and proofr...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,moby dick or the whale,herman melville,https://www.gutenberg.org/ebooks/2701,Free kindle book and epub digitized and proofr...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,romeo and juliet,william shakespeare,https://www.gutenberg.org/ebooks/1513,Free kindle book and epub digitized and proofr...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,pride and prejudice,jane austen,https://www.gutenberg.org/ebooks/1342,Free kindle book and epub digitized and proofr...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,alices adventures in wonderland,lewis carroll,https://www.gutenberg.org/ebooks/11,Free kindle book and epub digitized and proofr...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
df_final = pd.concat([df, df_api], ignore_index=True)

In [87]:
df_final.shape

(3397, 15)

In [88]:
mask = df_final["isbn13"].notna()
df_final.loc[mask] = df_final.loc[mask].drop_duplicates(subset=["isbn13"], keep="first")

In [94]:
df_final["unique_id"] = (
    df_final["title"].astype(str)
    + df_final["author"].astype(str)
).apply(hash)



In [96]:
df_final = df_final.drop_duplicates(subset=["isbn13", "unique_id"])


In [98]:
df_final.reset_index(drop=True, inplace=True)

In [99]:
df_final.shape


(3358, 16)

In [103]:
df_final[df_final["isbn13"].str.len() == 13].sample(5)


,title,author,url,description,subjects,categories,avg_rating,ratings_count,publisher,published_date,page_count,isbn10,isbn13,language,description_long,unique_id
3200,Creative Unity,rabindranath tagore,NaN,CIVILITY is beauty of behaviour. It requires f...,NaN,NaN,NaN,NaN,NaN,2020-06-07,94.0,None,9798651883196,NaN,NaN,-4982536598872506912
2632,The Fellowship of the Ring,john ronald reuel tolkien,NaN,The first book in the Lord of the rings trilogy.,NaN,Fiction,3.0,3.0,NaN,2002,580.0,0007149212,9780007149216,NaN,NaN,-9073404136409053649
2902,Financial Reporting and Analysis,lawrence revsine,NaN,Study Guide contains detailed chapter summarie...,NaN,Business & Economics,NaN,NaN,Prentice Hall,2004,444.0,0131430246,9780131430242,NaN,NaN,-7993234869449551299
3050,Physiology of Sport and Exercise,w. larry kenney,NaN,The leading textbook for undergraduate exercis...,NaN,Exercise,NaN,NaN,NaN,2012,621.0,1450421083,9781450421089,NaN,NaN,-4151878589102368571
3181,Starving Anonymous 2,kazu inabe,NaN,They found themselves in a sinister facility w...,NaN,Graphic novels,NaN,NaN,Kodansha America LLC,2018-04-10,211.0,1642121770,9781642121773,NaN,NaN,48999401218588238


In [104]:
df_final.shape


(3358, 16)

In [106]:
df_final.to_csv("books_final.csv", index=False, encoding="utf-8")

In [109]:
df_api.to_csv("books_api.csv", index=False, encoding="utf-8")

In [23]:
df = pd.read_csv("/home/rafael/Área de Trabalho/CUROS IRONHACK/SEMANA 10/get_your_book/raw/gutenberg_enriched.csv", engine="python")
df.head()

,title,author,url,description,subjects,categories,avg_rating,ratings_count,publisher,published_date,page_count,isbn10,isbn13,language,description_long
0,frankenstein or the modern prometheus,mary wollstonecraft shelley,https://www.gutenberg.org/ebooks/84,Free kindle book and epub digitized and proofr...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,moby dick or the whale,herman melville,https://www.gutenberg.org/ebooks/2701,Free kindle book and epub digitized and proofr...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,romeo and juliet,william shakespeare,https://www.gutenberg.org/ebooks/1513,Free kindle book and epub digitized and proofr...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,pride and prejudice,jane austen,https://www.gutenberg.org/ebooks/1342,Free kindle book and epub digitized and proofr...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,alices adventures in wonderland,lewis carroll,https://www.gutenberg.org/ebooks/11,Free kindle book and epub digitized and proofr...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#headers = {"User-Agent": "Mozilla/5.0"}
#base_url = "https://www.gutenberg.org"

url = "https://www.gutenberg.org/ebooks/search/?sort_order=downloads"
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

books = soup.select("li.booklink")


dataset = []
seen = set()

#num_pages = 100

#for i in range(num_pages):
    start_index = i * 25 + 1
    url = f"https://www.gutenberg.org/ebooks/search/?sort_order=downloads&start_index={start_index}"

    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        books = soup.select("li.booklink")
    except Exception as e:
        print(f"Error to load the page {i+1}: {e}")
        continue

    for book in books:
        try:
            title = book.select_one("span.title").text.strip()
            author = book.select_one("span.subtitle")
            author = author.text.strip() if author else "Unknown"
            link = urljoin(base_url, book.find("a")["href"])
        except:
            continue

        # to remove duplicates
        key = f"{title}|{author}"
        if key in seen:
            continue
        seen.add(key)

        # To open inner page
        try:
            r = requests.get(link, headers=headers)
            bs = BeautifulSoup(r.text, "html.parser")
    
            # Brief description
            desc = bs.find("meta", {"name": "description"})
            description = desc["content"] if desc else None
    
            # Bookshelves (categorias temáticas)
            subjects = [x.text.strip() for x in bs.select("td.bookshelves a")]

        except Exception as e:
            print(f"Error to open book: {title} → {e}")
            continue
            
            # ISBN — não existe para muitos (vamos pegar do Google Books depois)
            #isbn = None
    
        dataset.append({
            "title": title,
            "author": author,
            "url": link,
            "description": description,
            "subjects": subjects,
            #"isbn": isbn
        })

        # to save every 100 books
        if len(dataset) % 100 == 0:
            print(f"Saving {len(dataset)} books...")
            with open("gutenberg_books.csv", "w", newline='', encoding="utf-8") as f:
                keys = dataset[0].keys()
                writer = csv.DictWriter(f, fieldnames=keys)
                writer.writeheader()
                writer.writerows(dataset)
                
        time.sleep(0.3)

    print(f"Page {i+1} colected.")

with open("gutenberg_books.csv", "w", newline='', encoding="utf-8") as f:
    keys = dataset[0].keys()
    writer = csv.DictWriter(f, fieldnames=keys)
    writer.writerheader()
    writer.writerows(dataset)
    
len(dataset)


Page 1 colected.
Page 2 colected.
Page 3 colected.
Page 4 colected.
Saving 100 books...
Page 5 colected.
Page 6 colected.
Page 7 colected.
Page 8 colected.
Saving 200 books...
Page 9 colected.
Page 10 colected.
Page 11 colected.
Page 12 colected.
Saving 300 books...
Page 13 colected.
Page 14 colected.
Page 15 colected.
Page 16 colected.
Saving 400 books...
Page 17 colected.
Page 18 colected.
Page 19 colected.
Page 20 colected.
Saving 500 books...
Page 21 colected.
Page 22 colected.
Page 23 colected.
Page 24 colected.
Page 25 colected.
Saving 600 books...
Page 26 colected.
Page 27 colected.
Page 28 colected.
Page 29 colected.
Saving 700 books...
Page 30 colected.
Page 31 colected.
Page 32 colected.
Page 33 colected.
Saving 800 books...
Page 34 colected.
Page 35 colected.
Page 36 colected.
Page 37 colected.
Page 38 colected.
Saving 900 books...
Page 39 colected.
Page 40 colected.
Page 41 colected.
Page 42 colected.
Saving 1000 books...
Page 43 colected.
Page 44 colected.
Page 45 colected

KeyboardInterrupt: 